#### Задача 1 (10 баллов). 

Напишем мини-стратегическую игру, в которой игроку предлагается управлять городом (SimCity видели когда-нибудь?) У нас есть собственно класс "город", внутри города есть постройки разных типов, которые мы, как градоправитель, строим. Также у города есть атрибуты: уровень счастья жителей, уровень экологии, образования, миграции и так далее (на что вашей фантазии хватит). Постройки к этим атрибутам добавляют и прибавляют баллы. Можно завести тип постройки, которая на каждом ходе игры приносит в городской бюджет деньги (налоговая служба?.. заводы?). Постройки необходимо реализовать с помощью наследования, у некоторых построек могут быть свои методы, например, больница может устраивать медосмотры и вакцинации и поднимать этим уровень благосостояния жителей, а на заводе с рандомной вероятностью может случиться авария, которая понизит экологию. Не забудьте реализовать головной класс "игра", в котором будет метод play, можно в нем хранить и городской бюджет. 

Фантазия приветствуется: чем оригинальнее будет ваша игра, тем выше шанс, что я поставлю дополнительный балл. 

In [2]:
from random import randint, uniform

In [ ]:
class Building:
    def __init__(self, name, cost, maintenance, tax_income=0, happiness_boost=0, ecology_impact=0):
        self.name = name
        self.cost = cost
        self.maintenance = maintenance
        self.tax_income = tax_income
        self.happiness_boost = happiness_boost
        self.ecology_impact = ecology_impact
    
    def operate(self, city):
        city.budget += self.tax_income
        city.happiness += self.happiness_boost
        city.ecology -= self.ecology_impact
        city.budget -= self.maintenance
        print(f'{self.name}:\nПоступило в бюджет с налогов: {self.tax_income}\nОбслуживание')


In [ ]:
class Housing(Building):
    def __init__(self, name, cost, maintenance, residents):
        super().__init__(name, cost, maintenance) # импортируем конструктор родительского класса и добавляем новые атрибуты
        self.residents = residents
        self.tax_rate = 1.4
    
    def operate(self, city):
        income = self.residents * self.tax_rate # налоги с жильцов идут в бюджет
        city.budget += income
        city.population += self.residents
        city.happiness += 0.9
        resident = 'жителей'
        x = self.residents % 10
        xx = self.residents % 100
        if not 11 <= xx < 15:
            if x == 1:
                resident = 'житель'
            elif x in {2, 3, 4}:
                resident = 'жителя'
        print(f'Жилое здание {self.name} построено.\n{self.residents} {resident} добавлено.\nБюджет города: {city.budget}.')


In [ ]:
# магазины, супермаркеты, аптеки и т.п.
class Commerce(Building):
    def __init__(self, name, cost, maintenance):
        super().__init__(name, cost, maintenance)
        self.tax_income = uniform(100.0, 5000.0)
        self.tax_rate = 1.9

    def operate(self, city):
        income = self.tax_income * self.tax_rate
        city.budget += income
        city.happiness += 0.85
        print(f'Коммерческое здание {self.name} успешно построено.\nБюджет города: {city.budget}.\nУровень счастья жителей города повысился до {city.happiness}.')

In [ ]:
class Police(Building):
    def __init__(self, name, cost, maintenance, crime_reduction):
        super().__init__(name, cost, maintenance)
        self.crime_reduction = 1
    
    def operate(self, city):
        city.crime -= self.crime_reduction
        if city.crime < 0:
            city.crime = 0
        print(f'Уровень преступности снизился на {self.crime_reduction}.')

In [ ]:
class Hospital(Building):
    def operate(self, city):
        city.happiness += 0.6
        print(f'Лечение насиления успешно выполнено. Уровень счастья поднялся на 20.')

In [ ]:
class School(Building):
    def __init__(self, name, cost, maintenance):
        super().__init__(name, cost, maintenance) # импортируем конструктор родительского класса и добавляем атрибуты
        self.education = 1
    
    def operate(self, city):
        city.education += self.education
        city.happiness += 0.7
        print(f'Учебное заведение "{self.name}" успешно построено.\nУровень образования изменился до {self.education}.')

In [ ]:
# театры, кино, мероприятия
class Leisure(Building):
    # def __init__(self, name, cost, maintenance):
    #     super().__init__(name, cost, maintenance) # импортируем конструктор родительского класса и добавляем атрибуты
    #     self.education = 1
    
    def operate(self, city):
        city.happiness += 0.75
        print(f'Здание "{self.name}" успешно построено.\nУровень счастья жителей города изменился до {self.happiness}.')

In [ ]:
class Factory(Building):
    # def __init__(self, name, cost, maintenance):
    #     super().__init__(name, cost, maintenance)
    
    def operate(self, city):
        city.budget += 500
        city.ecology -= 10
        if uniform(0.0, 1.0) < 0.1:
            city.ecology -= 20
            print(f'Произошла авария. Уровень экологии снизился на 20.')
            # return
        print(f'Завод {self.name} успешно построен. Уровень экологии снизился на 10.\nБюджет города: {city.budget}.')

In [ ]:
# class TaxOffice(Building):
#     def collect_taxes(self, city):


In [ ]:
class City:
    # buildings = 0
    def __init__(self, name, budget):
        self.name = name
        self.budget = budget
        self.population = 0
        self.happiness = 0.5 # 0-1
        self.ecology = 0.5 # 0-100
        self.education = 0.5
        self.migration = 0 # информация о притоке или оттоке населения
        self.crime = 0
        self.buildings = [] # список построек
    
    def add_building(self, building):
        if self.budget >= building.cost:
            self.buildings.append(building)
            self.budget -= building.cost
            print(f'Здание {building.name} построено.\nБюджет города: {self.budget}.')
        else:
            print('Недостаточно средств бюджета для постройки здания.')
    
    def operate_buildings(self): # метод для обслуживания построек
        for building in self.buildings:
            building.operate(self)
            self.budget -= building.maintenance
            print(f'Обслуживание здания "{building.name}" стоило {building.maintenance}.\nБюджет города: {self.budget}.')
    
    def show_city_status(self):
        print(f'\nГород: {self.name}')
        print(f'Бюджет города {self.name}: {self.budget}')
        print(f'Население: {self.population}')
        print(f'Уровень счастья: {self.happiness}')
        print(f'Уровень экологии: {self.ecology}')
        print(f'Уровень образования: {self.education}')
        print(f'Уровень преступности: {self.crime}')
        print(f'Уровень миграции: {self.migration}')
        print(f'Постройки города: {', '.join(self.buildings)}')

In [ ]:
class Game:
    def __init__(self):
        self.city = City("Норт-Вилладж", 1000) # создаем экземпляр класса "Город" (сам город)
    
    def play(self):
        commands = [
            '1. Построить жилье',
            '2. Построить мед.учреждение',
            '3. Построить полицейский участок',
            '4. Построить коммерческое здание',
            '5. Построить больницу',
            '6. Построить учебное заведение',
            '7. Построить развлекательное здание',
            '8. Устроить развлекательное мероприятие в городе',
            '9. Показать статус города',
            '10. Выход']
        
        while True:
            print('\n'.join(commands))
            command = input('Какое действие хотите выполнить?')
            if command == '1':
                house = Housing('Жилой дом', 1000, 100, 20)
                self.city.add_building(house)
            elif command == '2':
                hospital = Hospital('Больница', )
                
